In [1]:
import networkx as nx
import obonet
import torch
import pandas as pd

/tmp/ipykernel_1197098/2362945497.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
GO_graph = obonet.read_obo("GNN/go-basic.obo")

go_edges = []
for u, v, data in GO_graph.edges(data=True):
    go_edges.append([u, v])
go_edges_df = pd.DataFrame(go_edges, columns=['Source', 'Target']).dropna()
print(go_edges_df.head())
col_name = ['GO']
for i in range(1,769):
  col_name.append('feature'+str(i))
go_features_df = pd.read_csv("GNN/go_terms_embeddings.csv", skiprows=1, names=col_name).dropna()
print(go_features_df.head())

       Source      Target
0  GO:0000001  GO:0048308
1  GO:0000001  GO:0048311
2  GO:0000002  GO:0007005
3  GO:0000003  GO:0008150
4  GO:0000006  GO:0005385
           GO  feature1  feature2  feature3  feature4  feature5  feature6  \
0  GO:0000001 -1.168093 -0.355214  0.265877 -0.710051  0.515028 -0.525165   
1  GO:0000002 -1.185879 -0.098765  0.388240 -0.295556  0.327296 -0.119842   
2  GO:0000003  0.063323 -0.199995  0.151511 -0.942141  0.109313  0.015316   
3  GO:0000005  0.163135  0.301527  0.219680  0.094342 -0.129769  0.225696   
4  GO:0000006 -0.641113 -0.541363  0.413941  0.699345  0.461507 -0.497388   

   feature7  feature8  feature9  ...  feature759  feature760  feature761  \
0 -0.186588 -0.161192  0.186984  ...   -1.350874   -0.991801   -0.648123   
1  0.399882 -0.035890  0.853417  ...   -1.086927   -0.842870   -0.385764   
2  0.633298  0.507875  0.665548  ...    0.174185    0.351648    0.138497   
3  0.357577  0.819992  0.852388  ...   -0.084025   -0.291103   -0.003621   
4

In [3]:
col_name = ['protein']
for i in range(1,769):
  col_name.append('feature'+str(i))
gene_features_df = pd.read_csv('GNN/gene_embedding_GeneLLM_2.csv', header=None, names=col_name).dropna()
#gene_features_df = gene_features_df.drop(gene_features_df.columns[0], axis=1)
print(gene_features_df.head())

col_name = ['Target', 'Source']
go_protein_df = pd.read_csv(
    "GNN/mart_export.txt", 
    skiprows=1, 
    names=col_name, 
    usecols=[1, 2] 
).dropna()
print(go_protein_df.head())

  protein  feature1  feature2  feature3  feature4  feature5  feature6  \
0     FES  0.339602 -0.030744 -0.901381  0.100888  0.886443  0.383596   
1  HADHA  -0.131799 -0.025745 -0.677301 -0.053545  0.971046  0.180315   
2  SLC7A7  0.385693 -0.070692 -0.847796 -0.022054  0.959772  0.085487   
3    LCK   0.650428  0.014479 -0.866163  0.053508  0.951529  0.269402   
4   HSPA2  0.322262  0.017484 -0.849302  0.046401  0.920429  0.463832   

   feature7  feature8  feature9  ...  feature759  feature760  feature761  \
0 -0.192082 -0.032063 -0.154869  ...   -0.549204   -0.856123    0.714672   
1 -0.028189 -0.077389 -0.095152  ...    0.927885   -0.817812    0.809631   
2  0.076455 -0.003006 -0.032268  ...    0.941094   -0.912443    0.789828   
3 -0.214788  0.045179 -0.506429  ...   -0.576739   -0.969558    0.916549   
4 -0.050414 -0.033398  0.387791  ...    0.387301   -0.860696    0.678607   

   feature762  feature763  feature764  feature765  feature766  feature767  \
0   -0.046649   -0.894424  

In [4]:
print(len(go_features_df))
go_features_df.rename(columns={'GO': 'protein'}, inplace=True)
combined_features = pd.concat([gene_features_df, go_features_df])

47595


In [5]:
combined_features

,protein,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,FES,0.339602,-0.030744,-0.901381,0.100888,0.886443,0.383596,-0.192082,-0.032063,-0.154869,...,-0.549204,-0.856123,0.714672,-0.046649,-0.894424,-0.001815,0.739485,0.015581,-0.023863,-0.022002
1,HADHA,-0.131799,-0.025745,-0.677301,-0.053545,0.971046,0.180315,-0.028189,-0.077389,-0.095152,...,0.927885,-0.817812,0.809631,-0.005827,-0.848839,0.024516,0.526404,-0.039926,-0.102787,-0.026980
2,SLC7A7,0.385693,-0.070692,-0.847796,-0.022054,0.959772,0.085487,0.076455,-0.003006,-0.032268,...,0.941094,-0.912443,0.789828,0.046979,-0.715636,0.085842,0.150494,0.025392,-0.066035,-0.028283
3,LCK,0.650428,0.014479,-0.866163,0.053508,0.951529,0.269402,-0.214788,0.045179,-0.506429,...,-0.576739,-0.969558,0.916549,-0.080332,-0.927649,-0.047398,0.741663,-0.000096,-0.096318,-0.056501
4,HSPA2,0.322262,0.017484,-0.849302,0.046401,0.920429,0.463832,-0.050414,-0.033398,0.387791,...,0.387301,-0.860696,0.678607,-0.060695,-0.945793,0.040472,0.831079,-0.001711,-0.079842,-0.011189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47590,GO:2001313,0.174428,0.194728,-0.284376,0.282102,-0.713190,-0.272055,0.121190,0.129901,-0.983496,...,0.500545,0.429651,-0.292929,-0.464941,-0.740187,0.179149,-0.960807,-0.746958,1.069112,-0.848182
47591,GO:2001314,0.025886,0.306214,-0.254303,0.253673,-0.533680,-0.269355,0.150939,-0.229323,-1.078991,...,0.042979,0.134560,-0.356661,-0.381828,-0.638338,0.077176,-0.788312,-0.683442,1.087031,-0.593092
47592,GO:2001315,0.027134,0.241391,-0.227353,0.317366,-0.726657,-0.197968,0.045653,0.038912,-0.954113,...,0.349853,0.370059,-0.144606,-0.493184,-0.655063,0.217335,-0.841272,-0.821077,1.036363,-0.836614
47593,GO:2001316,0.139543,0.028883,0.899480,0.152932,0.576852,0.330342,0.916943,0.012306,-0.020316,...,-0.354748,-0.083168,0.043640,-0.663565,0.543016,-0.652230,-1.427882,-0.985257,1.673561,0.109659


In [6]:
features_shuffled = combined_features.sample(frac=1).reset_index(drop=True)
features_shuffled

,protein,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature759,feature760,feature761,feature762,feature763,feature764,feature765,feature766,feature767,feature768
0,GO:0102030,0.245079,0.060955,0.177880,0.350591,-0.269516,0.093303,0.166691,-0.526311,-0.489648,...,-0.506199,0.249561,-0.208474,-0.410255,-0.444679,0.305353,-0.565994,-0.232176,0.268691,-0.316552
1,GO:0019222,0.343653,-0.020588,0.291593,-0.609495,-0.374338,0.275575,0.113448,0.142848,1.171809,...,0.000831,0.044567,-0.371669,0.325822,-0.259854,-0.027968,-0.134610,-0.576043,0.584789,0.513540
2,GO:0043651,-0.117396,0.459912,0.029887,0.131992,0.258211,0.491710,0.228968,-0.441609,0.125457,...,-0.801257,0.228723,-0.626664,-0.156087,-0.470884,-0.783440,-0.082460,0.570474,1.346214,0.938884
3,GO:0047715,-0.526277,0.565159,0.570244,-0.480896,-0.259288,0.733499,-0.427194,-0.378893,0.899117,...,-0.496169,0.358538,-0.724079,-0.695102,0.765133,-0.527925,-0.333353,0.358480,1.133885,0.674472
4,GO:0045228,1.044302,0.296786,0.807624,-0.958006,-1.118146,-0.197549,0.055317,-0.054696,-0.102629,...,0.070419,0.611316,-0.454071,-0.311416,-0.298850,0.454362,-0.907055,-0.584782,-0.082862,0.334650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62040,GO:0060225,0.168129,-0.268879,-0.208608,-0.353042,0.272458,-0.142894,0.751686,-0.469460,-0.328117,...,-0.141342,-0.410156,-0.555614,0.102004,0.093699,-1.112738,0.084535,-0.450197,-0.774177,-0.703010
62041,GO:0103077,-0.123654,0.493901,-0.475551,0.535989,0.046150,0.055551,0.558193,-0.602708,-0.754179,...,0.854525,0.204208,-0.182769,-1.125280,-0.247155,0.140447,-0.604479,-0.359852,0.039873,-0.030531
62042,KY,-0.270294,0.023677,-0.968183,-0.069810,0.976778,0.609132,-0.083191,-0.003226,-0.753098,...,0.895261,-0.888501,0.710627,-0.081105,-0.822674,0.019582,-0.087569,-0.125763,-0.045489,-0.025476
62043,TLE4,0.495169,-0.060087,-0.838078,0.018835,0.901561,0.086731,-0.140366,0.013444,0.305359,...,0.447866,-0.753992,0.674311,-0.064127,-0.874367,-0.014441,0.829651,0.066681,-0.166960,-0.009769


In [7]:
combined_edges = pd.concat([go_protein_df, go_edges_df])
combined_edges = combined_edges[['Source', 'Target']]
combined_edges

,Source,Target
0,GO:0030533,MT-TF
1,GO:0006412,MT-TF
4,GO:0003735,MT-RNR2
5,GO:0005840,MT-RNR2
6,GO:0030533,MT-TL1
...,...,...
83792,GO:2001317,GO:0034309
83793,GO:2001317,GO:0042181
83794,GO:2001317,GO:0120255
83795,GO:2001317,GO:1901362


In [8]:
nodes_in_features = set(combined_features['protein'])

filtered_edges_df = combined_edges[
    combined_edges['Source'].isin(nodes_in_features) & combined_edges['Target'].isin(nodes_in_features)
]


In [9]:
node_id_to_index = {node_id: i for i, node_id in enumerate(combined_features['protein'])}
# 确保edge_index是按照这个新的索引顺序排列的
source_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['Source']]
target_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['Target']]
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

In [10]:
edge_index

tensor([[24323, 17460, 27132,  ..., 62044, 62044, 62044],
        [ 2077,  2077,  2077,  ..., 52909, 55447, 62043]])

In [11]:
labels_df = pd.read_csv('GNN/new_labels.csv')
labels_df.rename(columns={'Gene name': 'protein'}, inplace=True)
labels_df.dropna()

,Unnamed: 0,protein,Solubility,Label,Word_Count,Count_Category
0,0,ERAP2,Membrane,0,117,0
1,1,ADAMTSL5,Soluble,1,28,1
2,2,TBC1D30,Membrane,0,55,0
3,3,KCNK18,Membrane,0,184,0
4,4,NDNF,Soluble,1,129,0
...,...,...,...,...,...,...
1374,1374,TRABD2B,Membrane,0,96,0
1375,1375,RPS9,Soluble,1,205,0
1376,1376,SLC22A16,Membrane,0,93,0
1377,1377,FBN3,Soluble,1,90,0


In [12]:
label_indices = [node_id_to_index[node_id] for node_id in labels_df['protein']]
print(label_indices)
num_nodes = len(combined_features)
labels = torch.full((num_nodes,), -1, dtype=torch.long)
for i, index in enumerate(labels_df['Label']):
    labels[label_indices[i]] = index

labels_tensor = torch.tensor(labels, dtype=torch.long)

[67, 81, 93, 94, 100, 116, 129, 142, 148, 149, 157, 167, 170, 202, 228, 236, 241, 243, 255, 258, 276, 281, 287, 294, 305, 319, 321, 334, 340, 369, 373, 375, 383, 384, 399, 402, 418, 447, 455, 474, 483, 484, 490, 498, 511, 518, 526, 558, 569, 585, 586, 587, 593, 594, 595, 596, 599, 615, 641, 653, 654, 664, 682, 736, 754, 762, 764, 773, 777, 781, 833, 852, 894, 895, 899, 915, 917, 938, 944, 957, 982, 983, 997, 1003, 1019, 1040, 1047, 1053, 1062, 1064, 1065, 1067, 1074, 1075, 1078, 1079, 1105, 1108, 1115, 1116, 1120, 1124, 1131, 1136, 1137, 1143, 1157, 1170, 1173, 1183, 1195, 1196, 1197, 1215, 1235, 1236, 1237, 1243, 1256, 1261, 1262, 1273, 1274, 1277, 1311, 1315, 1327, 1333, 1347, 1363, 1371, 1377, 1378, 1383, 1392, 1397, 1426, 1427, 1431, 1435, 1444, 1445, 1469, 1470, 1475, 1484, 1489, 1492, 1493, 1521, 1527, 1540, 1548, 1559, 1575, 1593, 1606, 1609, 1614, 1617, 1619, 1637, 1638, 1645, 1674, 1681, 1710, 1720, 1733, 1736, 1737, 1759, 1776, 1795, 1807, 1823, 1833, 1834, 1844, 1848, 1849, 

/tmp/ipykernel_1197098/2534729209.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)


In [13]:
features = combined_features.iloc[:, 1:].values
features_tensor = torch.tensor(features, dtype=torch.float)
features

array([[ 0.33960226, -0.03074448, -0.90138096, ...,  0.01558092,
        -0.02386307, -0.02200161],
       [-0.13179901, -0.02574519, -0.67730105, ..., -0.03992649,
        -0.10278717, -0.02697964],
       [ 0.38569278, -0.07069244, -0.8477959 , ...,  0.0253919 ,
        -0.06603534, -0.02828273],
       ...,
       [ 0.02713387,  0.24139147, -0.22735251, ..., -0.82107705,
         1.036363  , -0.83661443],
       [ 0.13954346,  0.02888298,  0.89947975, ..., -0.9852566 ,
         1.6735605 ,  0.10965873],
       [ 0.08306409,  0.09089889,  0.8885408 , ..., -0.79955566,
         1.5193683 ,  0.2632099 ]])

In [14]:
from torch_geometric.data import Data
data = Data(x=features_tensor, edge_index=edge_index, y=labels_tensor)

print("x:", data.x.shape, data.x.dtype)
print("edge_index:", data.edge_index.shape, data.edge_index.dtype)
print("labels:", data.y.shape, data.y.dtype)

x: torch.Size([62045, 768]) torch.float32
edge_index: torch.Size([2, 411251]) torch.int64
labels: torch.Size([62045]) torch.int64


In [15]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(nn.Module):
    def __init__(self, num_layers, in_dim, num_hidden, num_classes, heads, activation, dropout, negative_slope, residual):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        self.gat_layers = nn.ModuleList()
        self.activation = activation

        # Input projection (no residual)
        self.gat_layers.append(GATConv(
            in_dim, num_hidden, heads=heads[0],
            dropout=dropout, negative_slope=negative_slope, concat=True, add_self_loops=True))

        # Hidden layers
        for l in range(1, num_layers):
            # Due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATConv(
                num_hidden * heads[l-1], num_hidden, heads=heads[l],
                dropout=dropout, negative_slope=negative_slope, concat=True, add_self_loops=True))

        # Output projection
        self.gat_layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads=heads[-1],
            dropout=dropout, negative_slope=negative_slope, concat=False, add_self_loops=True))

    def forward(self, x, edge_index):
        h = x
        for l, layer in enumerate(self.gat_layers[:-1]):
            h = layer(h, edge_index)
            if self.activation:
                h = self.activation(h)
            if l < self.num_layers - 1:
                h = F.dropout(h, p=self.dropout, training=self.training)

        # Output projection
        logits = self.gat_layers[-1](h, edge_index)
        return logits

In [16]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Linear, ModuleList, Dropout

class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_layers, activation, dropout):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.convs = ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers - 2)])
        self.conv_last = GCNConv(hidden_dim, num_classes)
        self.activation = activation
        self.dropout = Dropout(dropout)
        self.num_layers = num_layers

    def forward(self, x, edge_index):
        # 输入层
        x = self.conv1(x, edge_index)
        x = self.activation(x)
        x = self.dropout(x)
        
        # 隐藏层
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.activation(x)
            x = self.dropout(x)

        # 输出层
        x = self.conv_last(x, edge_index)
        return x


In [17]:
from __future__ import division

import torch


def accuracy(pred, target):
    r"""Computes the accuracy of correct predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.

    :rtype: int
    """
    return (pred == target).sum().item() / target.numel()



def true_positive(pred, target, num_classes):
    r"""Computes the number of true positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target == i)).sum())

    return torch.tensor(out)



def true_negative(pred, target, num_classes):
    r"""Computes the number of true negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target != i)).sum())

    return torch.tensor(out)



def false_positive(pred, target, num_classes):
    r"""Computes the number of false positive predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred == i) & (target != i)).sum())

    return torch.tensor(out)



def false_negative(pred, target, num_classes):
    r"""Computes the number of false negative predictions.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`LongTensor`
    """
    out = []
    for i in range(num_classes):
        out.append(((pred != i) & (target == i)).sum())

    return torch.tensor(out)



def precision(pred, target, num_classes):
    r"""Computes the precision:
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FP}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fp = false_positive(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fp)
    out[torch.isnan(out)] = 0

    return out



def recall(pred, target, num_classes):
    r"""Computes the recall:
    :math:`\frac{\mathrm{TP}}{\mathrm{TP}+\mathrm{FN}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    tp = true_positive(pred, target, num_classes).to(torch.float)
    fn = false_negative(pred, target, num_classes).to(torch.float)

    out = tp / (tp + fn)
    out[torch.isnan(out)] = 0

    return out



def f1_score(pred, target, num_classes):
    r"""Computes the :math:`F_1` score:
    :math:`2 \cdot \frac{\mathrm{precision} \cdot \mathrm{recall}}
    {\mathrm{precision}+\mathrm{recall}}`.

    Args:
        pred (Tensor): The predictions.
        target (Tensor): The targets.
        num_classes (int): The number of classes.

    :rtype: :class:`Tensor`
    """
    prec = precision(pred, target, num_classes)
    rec = recall(pred, target, num_classes)

    score = 2 * (prec * rec) / (prec + rec)
    score[torch.isnan(score)] = 0

    return score

In [18]:
from sklearn.preprocessing import label_binarize
def train_model_scheduler(model, masked_features, labels, edge_index, optimizer, criterion, scheduler, train_mask):
    model.train()  # 设置模型为训练模
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index)  # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    scheduler.step(loss)
    return loss.item()

def train_model(model, masked_features, labels, edge_index, optimizer, criterion, train_mask):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index) # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    return loss.item()

def evaluate_model(model, features, labels, edge_index, mask):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 关闭梯度计算
        # 获取模型输出，这里假设输出已经是经过sigmoid的概率
        probabilities = model(features, edge_index)
        if probabilities.shape[1] == 2:  # 假设有两个输出（每个类一个概率）
            positive_probs = probabilities[mask, 1]  # 选择正类概率
        else:
            positive_probs = probabilities[mask]  # 如果只有一个输出，假设已经是正类概率
        val_f1 = torch.mean(f1_score(torch.argmax(probabilities[mask],dim=1), labels[mask], num_classes=2)).cpu().numpy()
        auc_score = roc_auc_score(labels[mask].cpu().numpy(), positive_probs.cpu().numpy())

    return val_f1, auc_score

In [19]:
from sklearn.metrics import roc_auc_score
import numpy as np
import random
from sklearn.model_selection import train_test_split
# 实例化模型
device = torch.device('cuda:1')
data = data.to(device)
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)


labeled_indices = label_indices
random.shuffle(labeled_indices)
num_labeled = len(labeled_indices)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)

276
tensor([False, False, False,  ..., False, False, False])


In [20]:
labeled_indices = label_indices
labeled_indices

[4166,
 3992,
 6804,
 9682,
 8075,
 7839,
 6377,
 498,
 8600,
 7278,
 11109,
 7083,
 11562,
 3839,
 5233,
 13314,
 10053,
 5298,
 142,
 1995,
 654,
 9800,
 10764,
 7266,
 13779,
 1426,
 599,
 11407,
 13214,
 13243,
 4676,
 2896,
 2819,
 7573,
 11002,
 6681,
 7783,
 7252,
 4935,
 6101,
 938,
 13001,
 9364,
 11000,
 2785,
 7640,
 915,
 2419,
 1898,
 11345,
 7892,
 5506,
 11286,
 13965,
 9910,
 243,
 10201,
 8031,
 14214,
 1493,
 11458,
 5242,
 6136,
 8733,
 7528,
 8272,
 6885,
 12304,
 373,
 1431,
 14117,
 13512,
 3159,
 2929,
 9416,
 10119,
 7375,
 6078,
 1136,
 7645,
 1143,
 10699,
 10700,
 14024,
 4232,
 944,
 4721,
 4025,
 1261,
 4780,
 852,
 2119,
 9908,
 6782,
 1614,
 1003,
 9325,
 13120,
 11125,
 13580,
 4323,
 8285,
 10890,
 11307,
 2283,
 12918,
 5253,
 6223,
 9639,
 6559,
 558,
 11210,
 7192,
 4760,
 10483,
 483,
 5898,
 14317,
 5586,
 5954,
 1235,
 5243,
 10172,
 2166,
 1736,
 5383,
 8012,
 5124,
 13931,
 3882,
 8748,
 129,
 13741,
 7930,
 6717,
 13936,
 5077,
 10159,
 6301,
 

In [21]:
import numpy as np
train_indices = labeled_indices[:num_train]
test_indices = labeled_indices[num_train:num_train+num_test]
print(train_indices)
print(test_indices)

[4166, 3992, 6804, 9682, 8075, 7839, 6377, 498, 8600, 7278, 11109, 7083, 11562, 3839, 5233, 13314, 10053, 5298, 142, 1995, 654, 9800, 10764, 7266, 13779, 1426, 599, 11407, 13214, 13243, 4676, 2896, 2819, 7573, 11002, 6681, 7783, 7252, 4935, 6101, 938, 13001, 9364, 11000, 2785, 7640, 915, 2419, 1898, 11345, 7892, 5506, 11286, 13965, 9910, 243, 10201, 8031, 14214, 1493, 11458, 5242, 6136, 8733, 7528, 8272, 6885, 12304, 373, 1431, 14117, 13512, 3159, 2929, 9416, 10119, 7375, 6078, 1136, 7645, 1143, 10699, 10700, 14024, 4232, 944, 4721, 4025, 1261, 4780, 852, 2119, 9908, 6782, 1614, 1003, 9325, 13120, 11125, 13580, 4323, 8285, 10890, 11307, 2283, 12918, 5253, 6223, 9639, 6559, 558, 11210, 7192, 4760, 10483, 483, 5898, 14317, 5586, 5954, 1235, 5243, 10172, 2166, 1736, 5383, 8012, 5124, 13931, 3882, 8748, 129, 13741, 7930, 6717, 13936, 5077, 10159, 6301, 67, 13559, 7958, 5079, 9225, 3985, 11523, 1645, 5228, 8560, 4249, 6819, 13321, 7130, 305, 9823, 11321, 2031, 13334, 2738, 13305, 2701, 3902

In [24]:
num_iterations = 100
count = 1 / num_iterations

# 总节点数
num_nodes = data.x.shape[0]
# 所有节点的索引
all_indices = np.arange(num_nodes)
mask_out = torch.ones(num_nodes, dtype=torch.bool)
# 将测试集索引处的掩码设为False
#mask_out[test_indices] = False
end = len(combined_features) - 47595
mask_out[-47595:] = False
mask_out[0:end] = False
# 使用掩码获取剩余的索引
remaining_indices = all_indices[mask_out]
complement_mask = ~mask_out
complement_mask[test_indices] = False

# 得到既不在 remaining_indices 也不在 test_indices 中的索引
complement_indices = all_indices[complement_mask]
print(len(complement_indices))

61769


In [25]:
num_nodes_out = len(complement_indices)
num_to_select = int(num_nodes_out * count)
for i in range(num_iterations+1):
    if i > 0:
    # 随机选择节点
        selected_indices = np.random.choice(complement_indices, num_to_select, replace=False)
        # 更新剩余节点列表
        complement_indices = np.setdiff1d(complement_indices, selected_indices)
    else:
        selected_indices = np.random.choice(complement_indices, 0, replace=False)
        complement_indices = np.setdiff1d(complement_indices, selected_indices)
    print("c:",len(complement_indices))
    # 保存到文件
    print("s:",len(selected_indices))
    np.save(f'DIVIDED_DATA/without_{i}%.npy', complement_indices)  # 修改保存路径

print("所有迭代的selected_indices和remaining_indices已保存到文件。")

c: 61769
s: 0
c: 61152
s: 617
c: 60535
s: 617
c: 59918
s: 617
c: 59301
s: 617
c: 58684
s: 617
c: 58067
s: 617
c: 57450
s: 617
c: 56833
s: 617
c: 56216
s: 617
c: 55599
s: 617
c: 54982
s: 617
c: 54365
s: 617
c: 53748
s: 617
c: 53131
s: 617
c: 52514
s: 617
c: 51897
s: 617
c: 51280
s: 617
c: 50663
s: 617
c: 50046
s: 617
c: 49429
s: 617
c: 48812
s: 617
c: 48195
s: 617
c: 47578
s: 617
c: 46961
s: 617
c: 46344
s: 617
c: 45727
s: 617
c: 45110
s: 617
c: 44493
s: 617
c: 43876
s: 617
c: 43259
s: 617
c: 42642
s: 617
c: 42025
s: 617
c: 41408
s: 617
c: 40791
s: 617
c: 40174
s: 617
c: 39557
s: 617
c: 38940
s: 617
c: 38323
s: 617
c: 37706
s: 617
c: 37089
s: 617
c: 36472
s: 617
c: 35855
s: 617
c: 35238
s: 617
c: 34621
s: 617
c: 34004
s: 617
c: 33387
s: 617
c: 32770
s: 617
c: 32153
s: 617
c: 31536
s: 617
c: 30919
s: 617
c: 30302
s: 617
c: 29685
s: 617
c: 29068
s: 617
c: 28451
s: 617
c: 27834
s: 617
c: 27217
s: 617
c: 26600
s: 617
c: 25983
s: 617
c: 25366
s: 617
c: 24749
s: 617
c: 24132
s: 617
c: 23515
s

In [26]:
class EarlyStopping:
    """早停机制，用于在验证损失停止改善时终止训练。"""
    def __init__(self, patience=200, verbose=False, delta=0):
        """
        参数:
            patience (int): 损失没有改善的迭代次数，在这之后训练将会被停止。
            verbose (bool): 如果为True，则打印一条消息表明早停被触发。
            delta (float): 损失的最小改变，被认为是改善。
            path (str): 最佳模型保存路径。
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_loss = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss, model, path):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, path)
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping triggered")
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model, path):
        '''保存模型当验证损失减少时'''
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).')

In [27]:
masked_features = features
masked_features[torch.tensor(complement_indices)] = 0
masked_features

array([[ 0.33960226, -0.03074448, -0.90138096, ...,  0.01558092,
        -0.02386307, -0.02200161],
       [-0.13179901, -0.02574519, -0.67730105, ..., -0.03992649,
        -0.10278717, -0.02697964],
       [ 0.38569278, -0.07069244, -0.8477959 , ...,  0.0253919 ,
        -0.06603534, -0.02828273],
       ...,
       [ 0.02713387,  0.24139147, -0.22735251, ..., -0.82107705,
         1.036363  , -0.83661443],
       [ 0.13954346,  0.02888298,  0.89947975, ..., -0.9852566 ,
         1.6735605 ,  0.10965873],
       [ 0.08306409,  0.09089889,  0.8885408 , ..., -0.79955566,
         1.5193683 ,  0.2632099 ]])

In [29]:
device = torch.device('cuda:1')
data = data.to(device)
results = []
num_epochs = 1000
# 加载数据
features = data.x
labels = data.y # 根据你的数据加载函数进行调整
num_iterations = 100
original_features = features.clone()
# 总共需要迭代的次数，这里以逐步增加5%为例，直到100%
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=3, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=50, verbose=True)
for i in range(num_iterations):
    # 加载这次迭代的selected_indices和remaining_indices
    selected_indices = np.load(f'DIVIDED_DATA/without_{i}%.npy')
    
    # 根据selected_indices和remaining_indices调整特征
    masked_features = features.clone()
    masked_features[torch.tensor(selected_indices)] = 0  # 假设features是一个PyTorch tensor
    
    early_stopping = EarlyStopping(patience=200, verbose=True, delta=0.00001)
    
    if i > 0:
        # 从上一个迭代保存的模型中加载参数
        model.load_state_dict(torch.load(f'G-G_DATA/G-G_model_WITHOUT{i-1}.pth'))
        # 重新初始化优化器和调度器
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
    
    # 模型训练和评估逻辑
    for epoch in range(num_epochs):
        train_loss = train_model_scheduler(model, masked_features, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
        test_f1, test_auc = evaluate_model(model, masked_features, data.y, data.edge_index, test_mask)
        #high_f1, high_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_high)
        #low_f1, low_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_low)
        if epoch % 50 == 0:  # 每10个epoch打印一次信息
            print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
        early_stopping(train_loss, model, i)
        if early_stopping.early_stop or epoch == num_epochs - 1:
            results.append({
                            'Train Loss': train_loss,
                            'F1': test_f1,
                            'AUC_score': test_auc
                        })
                            #'highinfo F1': high_f1,                                                            
                            #'highinfo AUC_score': high_auc,
                            #'lowinfo F1': low_f1,
                            #'lowinfo AUC_score': low_auc
            print("acc save")
            rate = 1 - count * (i + 1)
            rate = rate * 100
            print(f'{rate}% node features transform to 0: F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
            torch.save(model.state_dict(), f'G-G_DATA/G-G_model_WITHOUT{i}.pth')
            break

Epoch 0: Train Loss: 0.6931, Macro_F1: 0.3566, AUC_score: 0.5907
Validation loss decreased (0.693147 --> 0.693147).
Validation loss decreased (0.693119 --> 0.693119).
Epoch 50: Train Loss: 0.6931, Macro_F1: 0.3566, AUC_score: 0.5766
Epoch 00052: reducing learning rate of group 0 to 1.0000e-03.
Epoch 100: Train Loss: 0.6931, Macro_F1: 0.3566, AUC_score: 0.4154
Epoch 00103: reducing learning rate of group 0 to 2.0000e-04.
Epoch 150: Train Loss: 0.6931, Macro_F1: 0.3566, AUC_score: 0.4839
Epoch 00154: reducing learning rate of group 0 to 4.0000e-05.
Epoch 200: Train Loss: 0.6931, Macro_F1: 0.3566, AUC_score: 0.4878
Early stopping triggered
acc save
99.0% node features transform to 0: F1: 0.3566, AUC_score: 0.4917
Epoch 0: Train Loss: 0.6931, Macro_F1: 0.3566, AUC_score: 0.6031
Validation loss decreased (0.693114 --> 0.693114).
Validation loss decreased (0.693083 --> 0.693083).
Validation loss decreased (0.693065 --> 0.693065).
Validation loss decreased (0.693034 --> 0.693034).
Validation 

In [30]:
import json
import numpy as np
from numpy import array, float32
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.generic):
            return obj.item()
        return super(NumpyEncoder, self).default(obj)


# 保存为 JSON 文件
with open('GNN/zeroed_GO_network.json', 'w') as f:
    json.dump(results, f, cls=NumpyEncoder, indent=4)

In [20]:
print(num_train)
print(train_mask)

1103
tensor([False, False, False,  ..., False, False, False])


In [41]:
num_epochs = 2000
for epoch in range(num_epochs):
    train_loss = train_model_scheduler(model, data.x, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    test_f1, test_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_mask)
    
    if epoch % 50 == 0: 
        print(f'Epoch {epoch}: Train Loss:{train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')

Epoch 0: Train Loss:0.9409, Macro_F1: 0.3191, AUC_score: 0.8265
Epoch 50: Train Loss:0.2024, Macro_F1: 0.9344, AUC_score: 0.9660
Epoch 100: Train Loss:0.1137, Macro_F1: 0.9308, AUC_score: 0.9674
Epoch 150: Train Loss:0.0977, Macro_F1: 0.9417, AUC_score: 0.9679
Epoch 200: Train Loss:0.0781, Macro_F1: 0.9378, AUC_score: 0.9666
Epoch 250: Train Loss:0.0966, Macro_F1: 0.9237, AUC_score: 0.9645
Epoch 300: Train Loss:0.0590, Macro_F1: 0.9310, AUC_score: 0.9746
Epoch 350: Train Loss:0.0691, Macro_F1: 0.9378, AUC_score: 0.9726
Epoch 400: Train Loss:0.0365, Macro_F1: 0.9272, AUC_score: 0.9756
Epoch 450: Train Loss:0.0414, Macro_F1: 0.9165, AUC_score: 0.9768
Epoch 500: Train Loss:0.0392, Macro_F1: 0.9163, AUC_score: 0.9732
Epoch 550: Train Loss:0.0268, Macro_F1: 0.9201, AUC_score: 0.9774
Epoch 600: Train Loss:0.0295, Macro_F1: 0.9124, AUC_score: 0.9618
Epoch 650: Train Loss:0.0218, Macro_F1: 0.9232, AUC_score: 0.9732
Epoch 00686: reducing learning rate of group 0 to 1.0000e-03.
Epoch 700: Train 

In [20]:
num_layers=1
n_heads = 8
heads = ([n_heads] * num_layers) + [1]
heads

[8, 1]

In [44]:
from sklearn.metrics import roc_auc_score
import numpy as np
device = torch.device('cuda:1')
data = data.to(device)

model = GAT(num_layers=1, 
            in_dim=features.shape[1], 
            num_hidden=64, 
            num_classes=2, 
            heads = heads, 
            activation=F.elu, dropout=0.6, negative_slope=0.2, residual=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.001)#0.0005
loss_fn = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)

label_indices = torch.tensor(label_indices, dtype=torch.long)
print(label_indices)
# 随机打乱有标签的节点索引
labeled_indices = label_indices[torch.randperm(label_indices.size(0))]
#print(labeled_indices)
labeled_indices = label_indices

# 定义训练和测试集的大小
num_labeled = labeled_indices.size(0)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)
num_epochs = 1500
for epoch in range(num_epochs):
    train_loss = train_model_scheduler(model, data.x, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    #train_loss = train_model(model, data.x, data.y, data.edge_index, optimizer, loss_fn, train_mask)
    test_acc, test_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_mask)
    
    if epoch % 10 == 0: 
        print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_acc:.4f}, AUC_score: {test_auc:.4f}')

/tmp/ipykernel_3667016/3157765969.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_indices = torch.tensor(label_indices, dtype=torch.long)


tensor([2316, 6630, 4048,  ..., 5884, 5710, 7093])
276
tensor([False, False, False,  ..., False, False, False])
Epoch 0: Train Loss: 0.7221, Macro_F1: 0.6192, AUC_score: 0.7652
Epoch 10: Train Loss: 0.4897, Macro_F1: 0.8261, AUC_score: 0.9151
Epoch 20: Train Loss: 0.4087, Macro_F1: 0.8687, AUC_score: 0.9277
Epoch 30: Train Loss: 0.3778, Macro_F1: 0.8764, AUC_score: 0.9368
Epoch 40: Train Loss: 0.3309, Macro_F1: 0.8983, AUC_score: 0.9407
Epoch 50: Train Loss: 0.3248, Macro_F1: 0.9091, AUC_score: 0.9432
Epoch 60: Train Loss: 0.3187, Macro_F1: 0.9092, AUC_score: 0.9435
Epoch 70: Train Loss: 0.3118, Macro_F1: 0.9165, AUC_score: 0.9478
Epoch 80: Train Loss: 0.2821, Macro_F1: 0.9236, AUC_score: 0.9480
Epoch 90: Train Loss: 0.2945, Macro_F1: 0.9165, AUC_score: 0.9496
Epoch 100: Train Loss: 0.3178, Macro_F1: 0.9273, AUC_score: 0.9500
Epoch 110: Train Loss: 0.2957, Macro_F1: 0.9129, AUC_score: 0.9480
Epoch 120: Train Loss: 0.2959, Macro_F1: 0.9165, AUC_score: 0.9470
Epoch 130: Train Loss: 0.260

In [21]:
labeled_indices = label_indices
labeled_indices

[5144,
 4197,
 664,
 11362,
 236,
 8828,
 1834,
 3013,
 2986,
 10862,
 2476,
 2730,
 3943,
 1124,
 402,
 2455,
 4616,
 9836,
 8139,
 13295,
 6528,
 8356,
 13083,
 1327,
 7108,
 5146,
 116,
 13186,
 10443,
 894,
 8231,
 14052,
 7139,
 9937,
 2556,
 586,
 3454,
 4935,
 5351,
 9013,
 9639,
 5866,
 12272,
 3959,
 596,
 2308,
 2277,
 7065,
 9535,
 11609,
 10090,
 4040,
 9659,
 8865,
 4526,
 3707,
 1849,
 13582,
 8567,
 3797,
 6885,
 11340,
 321,
 6038,
 10151,
 13924,
 6168,
 11984,
 8653,
 5703,
 1170,
 5556,
 8632,
 1960,
 5589,
 5736,
 14325,
 6886,
 4417,
 4721,
 8716,
 5634,
 11345,
 3810,
 13978,
 3064,
 11022,
 14117,
 12954,
 1609,
 4169,
 8266,
 6782,
 4760,
 8595,
 9437,
 7125,
 9708,
 2937,
 7389,
 167,
 3751,
 10581,
 11707,
 6211,
 13130,
 8041,
 14012,
 3117,
 9126,
 81,
 2115,
 10479,
 10323,
 2140,
 9423,
 7769,
 5555,
 7047,
 228,
 11655,
 8290,
 9416,
 558,
 9098,
 9263,
 641,
 11746,
 7564,
 7390,
 11140,
 7665,
 3150,
 773,
 7073,
 5306,
 340,
 8720,
 3921,
 10934,
 3792

In [22]:
import numpy as np
train_indices = labeled_indices[:num_train]
test_indices = labeled_indices[num_train:num_train+num_test]
print(train_indices)
print(test_indices)

[5144, 4197, 664, 11362, 236, 8828, 1834, 3013, 2986, 10862, 2476, 2730, 3943, 1124, 402, 2455, 4616, 9836, 8139, 13295, 6528, 8356, 13083, 1327, 7108, 5146, 116, 13186, 10443, 894, 8231, 14052, 7139, 9937, 2556, 586, 3454, 4935, 5351, 9013, 9639, 5866, 12272, 3959, 596, 2308, 2277, 7065, 9535, 11609, 10090, 4040, 9659, 8865, 4526, 3707, 1849, 13582, 8567, 3797, 6885, 11340, 321, 6038, 10151, 13924, 6168, 11984, 8653, 5703, 1170, 5556, 8632, 1960, 5589, 5736, 14325, 6886, 4417, 4721, 8716, 5634, 11345, 3810, 13978, 3064, 11022, 14117, 12954, 1609, 4169, 8266, 6782, 4760, 8595, 9437, 7125, 9708, 2937, 7389, 167, 3751, 10581, 11707, 6211, 13130, 8041, 14012, 3117, 9126, 81, 2115, 10479, 10323, 2140, 9423, 7769, 5555, 7047, 228, 11655, 8290, 9416, 558, 9098, 9263, 641, 11746, 7564, 7390, 11140, 7665, 3150, 773, 7073, 5306, 340, 8720, 3921, 10934, 3792, 2957, 13901, 1197, 8360, 5510, 10424, 13422, 3319, 9225, 5178, 12151, 11289, 9115, 8942, 5243, 9561, 4185, 5899, 1256, 7456, 8749, 6438, 7

In [23]:
num_iterations = 100
count = 1 / num_iterations

# 总节点数
num_nodes = data.x.shape[0]
# 所有节点的索引
all_indices = np.arange(num_nodes)
mask_out = torch.ones(num_nodes, dtype=torch.bool)
# 将测试集索引处的掩码设为False
mask_out[test_indices] = False
mask_out[-47595:] = False
# 使用掩码获取剩余的索引
remaining_indices = all_indices[mask_out]
complement_mask = ~mask_out
complement_mask[test_indices] = False

# 得到既不在 remaining_indices 也不在 test_indices 中的索引
complement_indices = all_indices[complement_mask]

In [24]:
print(remaining_indices)
print(len(remaining_indices))
print(complement_indices)
print(len(complement_indices))

[    0     1     2 ... 14447 14448 14449]
14174
[14450 14451 14452 ... 62042 62043 62044]
47595


In [53]:
num_nodes_out = 47595
num_to_select = int(num_nodes_out * count)
for i in range(num_iterations):
    # 随机选择节点
    selected_indices = np.random.choice(complement_indices, num_to_select, replace=False)
    # 更新剩余节点列表
    complement_indices = np.setdiff1d(complement_indices, selected_indices)
    print("c:",len(complement_indices))
    # 保存到文件
    print("s:",len(selected_indices))
    np.save(f'DIVIDED_DATA/GO_{i}%.npy', complement_indices)  # 修改保存路径

print("所有迭代的selected_indices和remaining_indices已保存到文件。")

c: 47120
s: 475
c: 46645
s: 475
c: 46170
s: 475
c: 45695
s: 475
c: 45220
s: 475
c: 44745
s: 475
c: 44270
s: 475
c: 43795
s: 475
c: 43320
s: 475
c: 42845
s: 475
c: 42370
s: 475
c: 41895
s: 475
c: 41420
s: 475
c: 40945
s: 475
c: 40470
s: 475
c: 39995
s: 475
c: 39520
s: 475
c: 39045
s: 475
c: 38570
s: 475
c: 38095
s: 475
c: 37620
s: 475
c: 37145
s: 475
c: 36670
s: 475
c: 36195
s: 475
c: 35720
s: 475
c: 35245
s: 475
c: 34770
s: 475
c: 34295
s: 475
c: 33820
s: 475
c: 33345
s: 475
c: 32870
s: 475
c: 32395
s: 475
c: 31920
s: 475
c: 31445
s: 475
c: 30970
s: 475
c: 30495
s: 475
c: 30020
s: 475
c: 29545
s: 475
c: 29070
s: 475
c: 28595
s: 475
c: 28120
s: 475
c: 27645
s: 475
c: 27170
s: 475
c: 26695
s: 475
c: 26220
s: 475
c: 25745
s: 475
c: 25270
s: 475
c: 24795
s: 475
c: 24320
s: 475
c: 23845
s: 475
c: 23370
s: 475
c: 22895
s: 475
c: 22420
s: 475
c: 21945
s: 475
c: 21470
s: 475
c: 20995
s: 475
c: 20520
s: 475
c: 20045
s: 475
c: 19570
s: 475
c: 19095
s: 475
c: 18620
s: 475
c: 18145
s: 475
c: 17670

In [36]:
class EarlyStopping:
    """早停机制，用于在验证损失停止改善时终止训练。"""
    def __init__(self, patience=200, verbose=False, delta=0):
        """
        参数:
            patience (int): 损失没有改善的迭代次数，在这之后训练将会被停止。
            verbose (bool): 如果为True，则打印一条消息表明早停被触发。
            delta (float): 损失的最小改变，被认为是改善。
            path (str): 最佳模型保存路径。
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.best_loss = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_loss, model, path):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, path)
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping triggered")
        else:
            self.best_loss = val_loss
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0
    
    def save_checkpoint(self, val_loss, model, path):
        '''保存模型当验证损失减少时'''
        if self.verbose:
            print(f'Validation loss decreased ({self.best_loss:.6f} --> {val_loss:.6f}).')

In [25]:
features

array([[ 0.33960226, -0.03074448, -0.90138096, ...,  0.01558092,
        -0.02386307, -0.02200161],
       [-0.13179901, -0.02574519, -0.67730105, ..., -0.03992649,
        -0.10278717, -0.02697964],
       [ 0.38569278, -0.07069244, -0.8477959 , ...,  0.0253919 ,
        -0.06603534, -0.02828273],
       ...,
       [ 0.02713387,  0.24139147, -0.22735251, ..., -0.82107705,
         1.036363  , -0.83661443],
       [ 0.13954346,  0.02888298,  0.89947975, ..., -0.9852566 ,
         1.6735605 ,  0.10965873],
       [ 0.08306409,  0.09089889,  0.8885408 , ..., -0.79955566,
         1.5193683 ,  0.2632099 ]])

In [26]:
masked_features = features
masked_features[torch.tensor(complement_indices)] = 0
masked_features

array([[ 0.33960226, -0.03074448, -0.90138096, ...,  0.01558092,
        -0.02386307, -0.02200161],
       [-0.13179901, -0.02574519, -0.67730105, ..., -0.03992649,
        -0.10278717, -0.02697964],
       [ 0.38569278, -0.07069244, -0.8477959 , ...,  0.0253919 ,
        -0.06603534, -0.02828273],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [65]:
early_stopping = EarlyStopping(patience=300, verbose=True, delta=0.00001)
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
num_epochs = 2500

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)

# 模型训练和评估逻辑
for epoch in range(2000):
    train_loss = train_model_scheduler(model, masked_features, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    test_f1, test_auc = evaluate_model(model, masked_features, data.y, data.edge_index, test_mask)

    if epoch % 50 == 0:  # 每10个epoch打印一次信息
        print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
    early_stopping(train_loss, model, i)
    if early_stopping.early_stop or epoch == num_epochs - 1:
        results.append({
                        'Train Loss': train_loss,
                        'F1': test_f1,
                        'AUC_score': test_auc
                    })
        print("acc save")
        rate = 1 - count * (i + 1)
        rate = rate * 100
        print(f'{rate}% node features transform to 0: F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
        torch.save(model.state_dict(), f'G-G_DATA/G-G_model_WITHOUT{i}.pth')
        break

Epoch 0: Train Loss: 0.6916, Macro_F1: 0.3381, AUC_score: 0.5823
Validation loss decreased (0.691620 --> 0.691620).
Validation loss decreased (0.690761 --> 0.690761).
Validation loss decreased (0.690443 --> 0.690443).
Validation loss decreased (0.690369 --> 0.690369).
Validation loss decreased (0.690162 --> 0.690162).
Validation loss decreased (0.689873 --> 0.689873).
Validation loss decreased (0.688683 --> 0.688683).
Validation loss decreased (0.687966 --> 0.687966).
Validation loss decreased (0.687802 --> 0.687802).
Validation loss decreased (0.686706 --> 0.686706).
Validation loss decreased (0.686456 --> 0.686456).
Validation loss decreased (0.686115 --> 0.686115).
Validation loss decreased (0.685075 --> 0.685075).
Epoch 50: Train Loss: 0.6859, Macro_F1: 0.6323, AUC_score: 0.6673
Validation loss decreased (0.684499 --> 0.684499).
Validation loss decreased (0.682833 --> 0.682833).
Validation loss decreased (0.682796 --> 0.682796).
Validation loss decreased (0.682426 --> 0.682426).
Va

In [27]:
masked_features = features
masked_features[torch.tensor(remaining_indices)] = 0
masked_features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
early_stopping = EarlyStopping(patience=300, verbose=True, delta=0.00001)
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
num_epochs = 2500
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)

# 模型训练和评估逻辑
for epoch in range(2000):
    train_loss = train_model_scheduler(model, masked_features, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
    test_f1, test_auc = evaluate_model(model, masked_features, data.y, data.edge_index, test_mask)

    if epoch % 50 == 0:  # 每10个epoch打印一次信息
        print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
    early_stopping(train_loss, model, i)
    if early_stopping.early_stop or epoch == num_epochs - 1:
        results.append({
                        'Train Loss': train_loss,
                        'F1': test_f1,
                        'AUC_score': test_auc
                    })
        print("acc save")
        print(f'F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
        #torch.save(model.state_dict(), f'G-G_DATA/G-G_model_WITHOUT{i}.pth')
        break

Epoch 0: Train Loss: 0.8644, Macro_F1: 0.6356, AUC_score: 0.5760
Validation loss decreased (0.864432 --> 0.864432).
Validation loss decreased (0.683808 --> 0.683808).
Validation loss decreased (0.530536 --> 0.530536).
Validation loss decreased (0.459387 --> 0.459387).
Validation loss decreased (0.434727 --> 0.434727).
Validation loss decreased (0.424424 --> 0.424424).
Validation loss decreased (0.394828 --> 0.394828).
Validation loss decreased (0.345164 --> 0.345164).
Validation loss decreased (0.317922 --> 0.317922).
Validation loss decreased (0.312071 --> 0.312071).
Validation loss decreased (0.286887 --> 0.286887).
Validation loss decreased (0.274502 --> 0.274502).
Validation loss decreased (0.270713 --> 0.270713).
Validation loss decreased (0.244378 --> 0.244378).
Validation loss decreased (0.229917 --> 0.229917).
Validation loss decreased (0.229369 --> 0.229369).
Validation loss decreased (0.226182 --> 0.226182).
Validation loss decreased (0.204944 --> 0.204944).
Validation loss d

In [29]:
labels_df

,Unnamed: 0,protein,Solubility,Label,Word_Count,Count_Category
0,0,ERAP2,Membrane,0,117,0
1,1,ADAMTSL5,Soluble,1,28,1
2,2,TBC1D30,Membrane,0,55,0
3,3,KCNK18,Membrane,0,184,0
4,4,NDNF,Soluble,1,129,0
...,...,...,...,...,...,...
1374,1374,TRABD2B,Membrane,0,96,0
1375,1375,RPS9,Soluble,1,205,0
1376,1376,SLC22A16,Membrane,0,93,0
1377,1377,FBN3,Soluble,1,90,0


In [32]:
highinfo_indices = [
    node_id_to_index[node_id]
    for node_id, weight in zip(labels_df['protein'], labels_df['Count_Category'])
    if weight == 0
]
print(highinfo_indices)
lowinfo_indices = [
    node_id_to_index[node_id]
    for node_id, weight in zip(labels_df['protein'], labels_df['Count_Category'])
    if weight == 1
]
print(lowinfo_indices)

[67, 93, 94, 100, 116, 129, 142, 148, 157, 167, 228, 236, 241, 255, 276, 287, 294, 321, 334, 340, 369, 373, 375, 383, 384, 399, 402, 418, 447, 474, 483, 490, 498, 511, 518, 526, 558, 569, 586, 587, 593, 594, 595, 596, 599, 615, 641, 654, 664, 682, 736, 754, 764, 777, 833, 852, 894, 895, 899, 915, 917, 938, 944, 957, 982, 983, 997, 1003, 1019, 1040, 1053, 1062, 1064, 1065, 1067, 1074, 1075, 1078, 1079, 1105, 1108, 1115, 1116, 1120, 1124, 1131, 1136, 1137, 1143, 1157, 1170, 1173, 1183, 1195, 1196, 1197, 1215, 1235, 1236, 1237, 1243, 1256, 1261, 1273, 1274, 1277, 1311, 1315, 1327, 1333, 1347, 1363, 1377, 1378, 1392, 1397, 1427, 1444, 1445, 1469, 1470, 1475, 1484, 1489, 1492, 1493, 1521, 1548, 1559, 1575, 1593, 1609, 1614, 1617, 1619, 1637, 1638, 1674, 1681, 1720, 1733, 1736, 1737, 1759, 1776, 1795, 1807, 1833, 1834, 1844, 1849, 1852, 1854, 1882, 1893, 1895, 1898, 1914, 1917, 1923, 1960, 1970, 1972, 1978, 1986, 1995, 2015, 2031, 2046, 2048, 2071, 2096, 2097, 2101, 2109, 2115, 2117, 2140, 2

In [33]:
print(len(highinfo_indices))
print(len(lowinfo_indices))

1203
176


In [34]:
high_indices = torch.tensor(highinfo_indices, dtype=torch.long)
high_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
high_mask[high_indices] = True
test_high = high_mask & test_mask
low_indices = torch.tensor(lowinfo_indices, dtype=torch.long)
low_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
low_mask[low_indices] = True
test_low = low_mask & test_mask

In [45]:
true_count = sum(test_high)
true_count

tensor(240)

In [46]:
true_count = sum(test_low)
true_count

tensor(36)

In [39]:
device = torch.device('cuda:1')
data = data.to(device)
results = []
# 加载数据
features = data.x
labels = data.y # 根据你的数据加载函数进行调整
num_iterations = 50
original_features = features.clone()
# 总共需要迭代的次数，这里以逐步增加5%为例，直到100%
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
num_epochs = 2500
for i in range(100):
    # 加载这次迭代的selected_indices和remaining_indices
    selected_indices = np.load(f'DIVIDED_DATA/GO_{i}%.npy')
    
    # 根据selected_indices和remaining_indices调整特征
    masked_features = features.clone()
    masked_features[torch.tensor(selected_indices)] = 0  # 假设features是一个PyTorch tensor
    
    early_stopping = EarlyStopping(patience=300, verbose=True, delta=0.00001)
    
    if i > 0:
        # 从上一个迭代保存的模型中加载参数
        model.load_state_dict(torch.load(f'G-G_DATA/G-G_model_WITHOUT{i-1}.pth'))
        # 重新初始化优化器和调度器
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
    
    # 模型训练和评估逻辑
    for epoch in range(num_epochs):
        train_loss = train_model_scheduler(model, masked_features, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
        test_f1, test_auc = evaluate_model(model, masked_features, data.y, data.edge_index, test_mask)
        high_f1, high_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_high)
        low_f1, low_auc = evaluate_model(model, data.x, data.y, data.edge_index, test_low)
        if epoch % 50 == 0:  # 每10个epoch打印一次信息
            print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
        early_stopping(train_loss, model, i)
        if early_stopping.early_stop or epoch == num_epochs - 1:
            results.append({
                            'Train Loss': train_loss,
                            'F1': test_f1,
                            'AUC_score': test_auc,
                            'highinfo F1': high_f1,
                            'highinfo AUC_score': high_auc,
                            'lowinfo F1': low_f1,
                            'lowinfo AUC_score': low_auc
                        })
            print("acc save")
            rate = 1 - count * (i + 1)
            rate = rate * 100
            print(f'{rate}% node features transform to 0: F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
            torch.save(model.state_dict(), f'G-G_DATA/G-G_model_WITHOUT{i}.pth')
            break

Epoch 0: Train Loss: 0.6922, Macro_F1: 0.5085, AUC_score: 0.7057
Validation loss decreased (0.692233 --> 0.692233).
Validation loss decreased (0.688132 --> 0.688132).
Validation loss decreased (0.685286 --> 0.685286).
Validation loss decreased (0.682163 --> 0.682163).
Validation loss decreased (0.677474 --> 0.677474).
Validation loss decreased (0.677233 --> 0.677233).
Validation loss decreased (0.673267 --> 0.673267).
Validation loss decreased (0.667494 --> 0.667494).
Validation loss decreased (0.666953 --> 0.666953).
Validation loss decreased (0.661544 --> 0.661544).
Validation loss decreased (0.661447 --> 0.661447).
Validation loss decreased (0.660158 --> 0.660158).
Validation loss decreased (0.654400 --> 0.654400).
Validation loss decreased (0.653681 --> 0.653681).
Validation loss decreased (0.653354 --> 0.653354).
Validation loss decreased (0.644886 --> 0.644886).
Validation loss decreased (0.643683 --> 0.643683).
Validation loss decreased (0.641803 --> 0.641803).
Validation loss d

In [40]:
results

[{'Train Loss': 0.36859965324401855,
  'F1': array(0.83690304, dtype=float32),
  'AUC_score': 0.9255067122927086,
  'highinfo F1': array(0.77232796, dtype=float32),
  'highinfo AUC_score': 0.8329036304090308,
  'lowinfo F1': array(0.9166023, dtype=float32),
  'lowinfo AUC_score': 0.9197530864197532},
 {'Train Loss': 0.2899118959903717,
  'F1': array(0.8477541, dtype=float32),
  'AUC_score': 0.9310871281916294,
  'highinfo F1': array(0.8043443, dtype=float32),
  'highinfo AUC_score': 0.8786844122360811,
  'lowinfo F1': array(0.8885449, dtype=float32),
  'lowinfo AUC_score': 0.9320987654320988},
 {'Train Loss': 0.22125475108623505,
  'F1': array(0.8219378, dtype=float32),
  'AUC_score': 0.8960252698078441,
  'highinfo F1': array(0.76496243, dtype=float32),
  'highinfo AUC_score': 0.8899728241934362,
  'lowinfo F1': array(0.8017309, dtype=float32),
  'lowinfo AUC_score': 0.904320987654321},
 {'Train Loss': 0.0839753970503807,
  'F1': array(0.9273493, dtype=float32),
  'AUC_score': 0.96535

In [41]:
import json
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.generic):
            return obj.item()
        return super(NumpyEncoder, self).default(obj)

# 保存为 JSON 文件
with open('GNN/results_GP_data_2.json', 'w') as f:
    json.dump(results, f, cls=NumpyEncoder, indent=4)


In [32]:

for i in range(20):
    # 加载这次迭代的selected_indices和remaining_indices
    i = i+20
    selected_indices = np.load(f'DIVIDED_DATA/GO_{i}%.npy')
    
    # 根据selected_indices和remaining_indices调整特征
    masked_features = features.clone()
    masked_features[torch.tensor(selected_indices)] = 0  # 假设features是一个PyTorch tensor
    
    early_stopping = EarlyStopping(patience=300, verbose=True, delta=0.00001)
    
    if i > 0:
        # 从上一个迭代保存的模型中加载参数
        model.load_state_dict(torch.load(f'G-G_DATA/G-G_model_WITHOUT{i-1}.pth'))
        # 重新初始化优化器和调度器
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
    
    # 模型训练和评估逻辑
    for epoch in range(num_epochs):
        train_loss = train_model_scheduler(model, masked_features, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
        test_f1, test_auc = evaluate_model(model, masked_features, data.y, data.edge_index, test_mask)

        if epoch % 50 == 0:  # 每10个epoch打印一次信息
            print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
        early_stopping(train_loss, model, i)
        if early_stopping.early_stop or epoch == num_epochs - 1:
            results.append({
                            'Train Loss': train_loss,
                            'F1': test_f1,
                            'AUC_score': test_auc
                        })
            print("acc save")
            rate = 1 - count * (i + 1)
            rate = rate * 100
            print(f'{rate}% node features transform to 0: F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
            torch.save(model.state_dict(), f'G-G_DATA/G-G_model_WITHOUT{i}.pth')
            break

Epoch 0: Train Loss: 0.0167, Macro_F1: 0.9130, AUC_score: 0.9664
Validation loss decreased (0.016663 --> 0.016663).
Validation loss decreased (0.012255 --> 0.012255).
Validation loss decreased (0.011488 --> 0.011488).
Validation loss decreased (0.010566 --> 0.010566).
Validation loss decreased (0.009436 --> 0.009436).
Epoch 50: Train Loss: 0.0095, Macro_F1: 0.9203, AUC_score: 0.9721
Validation loss decreased (0.008950 --> 0.008950).
Validation loss decreased (0.007095 --> 0.007095).
Epoch 100: Train Loss: 0.0102, Macro_F1: 0.9094, AUC_score: 0.9687
Epoch 150: Train Loss: 0.0117, Macro_F1: 0.9275, AUC_score: 0.9722
Epoch 00196: reducing learning rate of group 0 to 2.0000e-04.
Epoch 200: Train Loss: 0.0110, Macro_F1: 0.9239, AUC_score: 0.9698
Epoch 250: Train Loss: 0.0155, Macro_F1: 0.9239, AUC_score: 0.9703
Epoch 00297: reducing learning rate of group 0 to 4.0000e-05.
Epoch 300: Train Loss: 0.0150, Macro_F1: 0.9130, AUC_score: 0.9701
Epoch 350: Train Loss: 0.0107, Macro_F1: 0.9167, AUC_

In [31]:
device = torch.device('cuda:1')
data = data.to(device)
results_2 = []
# 加载数据
features = data.x
labels = data.y # 根据你的数据加载函数进行调整
num_iterations = 1
original_features = features.clone()
# 总共需要迭代的次数，这里以逐步增加5%为例，直到100%
model = GCN(num_features=data.x.shape[1], hidden_dim=64, num_classes=2, num_layers=2, activation=F.relu, dropout=0.5)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
num_epochs = 2500
for i in range(num_iterations):
    # 加载这次迭代的selected_indices和remaining_indices
    selected_indices = np.load(f'DIVIDED_DATA/selected_indices_iteration_{i}.npy')
    remaining_indices = np.load(f'DIVIDED_DATA/remaining_indices_after_iteration_{i}.npy')
    
    # 根据selected_indices和remaining_indices调整特征
    masked_features = features.clone()
    masked_features[torch.tensor(remaining_indices)] = 0  # 假设features是一个PyTorch tensor
    
    early_stopping = EarlyStopping(patience=300, verbose=True, delta=0.00001)
    
    if i > 0:
        # 从上一个迭代保存的模型中加载参数
        model.load_state_dict(torch.load(f'G-G_DATA/G-G_model_WITHOUT_2{i-1}.pth'))
        # 重新初始化优化器和调度器
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)
    
    # 模型训练和评估逻辑
    for epoch in range(num_epochs):
        train_loss = train_model_scheduler(model, masked_features, data.y, data.edge_index, optimizer, loss_fn, scheduler, train_mask)
        test_f1, test_auc = evaluate_model(model, masked_features, data.y, data.edge_index, test_mask)

        if epoch % 50 == 0:  # 每10个epoch打印一次信息
            print(f'Epoch {epoch}: Train Loss: {train_loss:.4f}, Macro_F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
        early_stopping(train_loss, model, i)
        if early_stopping.early_stop or epoch == num_epochs - 1:
            results_2.append({
                            'Train Loss': train_loss,
                            'F1': test_f1,
                            'AUC_score': test_auc
                        })
            print("acc save")
            rate = 1 - count * (i + 1)
            rate = rate * 100
            print(f'{rate}% node features transform to 0: F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')
            torch.save(model.state_dict(), f'G-G_DATA/G-G_model_WITHOUT_2{i}.pth')
            break

Epoch 0: Train Loss: 0.8002, Macro_F1: 0.7219, AUC_score: 0.8225
Validation loss decreased (0.800181 --> 0.800181).
Validation loss decreased (0.579283 --> 0.579283).
Validation loss decreased (0.484609 --> 0.484609).
Validation loss decreased (0.428152 --> 0.428152).
Validation loss decreased (0.417531 --> 0.417531).
Validation loss decreased (0.416901 --> 0.416901).
Validation loss decreased (0.392682 --> 0.392682).
Validation loss decreased (0.374649 --> 0.374649).
Validation loss decreased (0.374201 --> 0.374201).
Validation loss decreased (0.363151 --> 0.363151).
Validation loss decreased (0.310136 --> 0.310136).
Validation loss decreased (0.294451 --> 0.294451).
Validation loss decreased (0.288247 --> 0.288247).
Validation loss decreased (0.261868 --> 0.261868).
Validation loss decreased (0.251904 --> 0.251904).
Validation loss decreased (0.247174 --> 0.247174).
Validation loss decreased (0.238913 --> 0.238913).
Validation loss decreased (0.235150 --> 0.235150).
Validation loss d

In [54]:
results_2

[{'Train Loss': 0.4924525320529938,
  'F1': array(0.8043854, dtype=float32),
  'AUC_score': 0.8161652496184871}]